In [3]:
from transformers import *
import os
import pandas as pd

I0511 12:35:47.221462 139756695570176 file_utils.py:41] PyTorch version 1.4.0 available.
I0511 12:35:48.351346 139756695570176 file_utils.py:57] TensorFlow version 2.1.0 available.


In [65]:
tokenizer = AlbertTokenizer.from_pretrained(
            pretrained_model_name_or_path="transformers_vocab/albert-large-v2-spiece.model",
            lowercase=True,
            add_prefix_space=True
        )

def get_prediction(tweet, selected_text):
    
    tweet = " ".join(tweet.split())
    selected_text = " ".join(selected_text.split())
    
    selected_text_copy = "".join(selected_text.split())
    tweet_copy = "".join(tweet.split())

    len_st = len(selected_text_copy)
    idx0 = None
    idx1 = None

    for ind in (i for i, e in enumerate(tweet_copy) if e == selected_text_copy[0]):
        if tweet_copy[ind: ind + len_st] == selected_text_copy:
            idx0 = ind
            idx1 = ind + len_st - 1
            break
    
    char_targets = [0] * len(tweet_copy)
    if idx0 != None and idx1 != None:
        for ct in range(idx0, idx1 + 1):
            char_targets[ct] = 1
            
    input_ids_orig = tokenizer.encode(tweet)
    input_ids_orig = input_ids_orig[1:-1]
    
    tweet_offsets = []
    idx = 0
    for t in input_ids_orig:
        w = tokenizer.decode([t])
        tweet_offsets.append((idx, idx + len(w)))
        idx += len(w)
        
    target_idx = []
    for j, (offset1, offset2) in enumerate(tweet_offsets):
        if sum(char_targets[offset1: offset2]) > 0:
            target_idx.append(j)
            
    targets_start = target_idx[0]
    targets_end = target_idx[-1]
    tweet_copy = "".join(tweet.split())

    # ... i kinda lost respect  . i kinda lost respect
    prediction = ""
    for ix in range(targets_start, targets_end + 1):
        prediction += tweet_copy[tweet_offsets[ix][0]: tweet_offsets[ix][1]]
        if (ix + 1) < len(tweet_offsets) and tweet_offsets[ix][1] < tweet_offsets[ix + 1][0]:
            prediction += " "
            
    len_prediction = len(prediction)

    for ind in (i for i, e in enumerate(tweet) if e == prediction[0]):

        tweet_sub_sentence = "".join(tweet[ind:].split())

        if tweet_sub_sentence[:len_prediction] == prediction:
            orig_idx0 = ind
            break

    for ind in range(orig_idx0, len(tweet)):

        if "".join(tweet[orig_idx0: ind+1].split()) == prediction:
            filtered_output = tweet[orig_idx0: ind+1]
            break
            
    return filtered_output
    

I0512 00:00:30.669103 139756695570176 tokenization_utils.py:420] Model name 'transformers_vocab/albert-large-v2-spiece.model' not found in model shortcut name list (albert-base-v1, albert-large-v1, albert-xlarge-v1, albert-xxlarge-v1, albert-base-v2, albert-large-v2, albert-xlarge-v2, albert-xxlarge-v2). Assuming 'transformers_vocab/albert-large-v2-spiece.model' is a path, a model identifier, or url to a directory containing tokenizer files.
W0512 00:00:30.670040 139756695570176 tokenization_utils.py:432] Calling AlbertTokenizer.from_pretrained() with the path to a single file or url is deprecated
I0512 00:00:30.670568 139756695570176 tokenization_utils.py:502] loading file transformers_vocab/albert-large-v2-spiece.model


In [69]:
tokenizer = AlbertTokenizer.from_pretrained(
            pretrained_model_name_or_path="transformers_vocab/albert-large-v2-spiece.model",
            lowercase=True,
            add_prefix_space=True
        )

def get_prediction_v2(tweet, selected_text):
    
    tweet = " ".join(tweet.split())
    selected_text = " ".join(selected_text.split())
    
    len_st = len(selected_text)
    idx0 = None
    idx1 = None
    
    # get char idx
    for ind in (i for i, e in enumerate(tweet) if e == selected_text[0]):
        if tweet[ind: ind + len_st] == selected_text:
            idx0 = ind
            idx1 = ind + len_st - 1
            break
    
    # get char mask
    char_targets = [0] * len(tweet)
    if idx0 != None and idx1 != None:
        for ct in range(idx0, idx1 + 1):
            char_targets[ct] = 1
    
    # get word offsets
    tweet_offsets = []
    idx = 0
    
    for word in tweet.split():
        tweet_offsets.append((idx, idx + len(word)))
        # add " ", we have done " ".join() before so only one " " between words
        idx += len(word) + 1
    
    # get word idx
    target_idx = []
    for j, (offset1, offset2) in enumerate(tweet_offsets):
    
        if sum(char_targets[offset1: offset2]) > 0:
            target_idx.append(j)
            
    targets_start = target_idx[0]
    targets_end = target_idx[-1]
    
#     print(tweet.split()[targets_start: targets_end+1])
    
    # get sub_token idx
    tweet_word = tweet.split()
    selected_text_word = selected_text.split()
    word_to_first_token_index = {}
    word_to_last_token_index = {}
    token_to_word_index = {}
    sub_token_idx = 0
    
    for (i, word) in enumerate(tweet_word):
        
        sub_tokens = tokenizer.tokenize(word)
        
        word_to_first_token_index[i] = sub_token_idx
        
        for sub_token in sub_tokens:
            
            token_to_word_index[sub_token_idx] = i
            sub_token_idx += 1
            
        word_to_last_token_index[i] = sub_token_idx - 1
        
    targets_start_token = word_to_first_token_index[targets_start]
    targets_end_token = word_to_first_token_index[targets_end]
            
    encoded = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(tweet))
    
#     return tweet[token_to_word_index[targets_start_token]: token_to_word_index[targets_end_token]]
    
    print(tweet[token_to_word_index[targets_start_token]: token_to_word_index[targets_end_token]], selected_text)

tweet = "S`ok, trying to plot alternatives as we speak *sigh*"
selected_text = "*sigh*"
get_prediction_v2(tweet, selected_text)

I0512 18:29:05.413665 139756695570176 tokenization_utils.py:420] Model name 'transformers_vocab/albert-large-v2-spiece.model' not found in model shortcut name list (albert-base-v1, albert-large-v1, albert-xlarge-v1, albert-xxlarge-v1, albert-base-v2, albert-large-v2, albert-xlarge-v2, albert-xxlarge-v2). Assuming 'transformers_vocab/albert-large-v2-spiece.model' is a path, a model identifier, or url to a directory containing tokenizer files.
W0512 18:29:05.414740 139756695570176 tokenization_utils.py:432] Calling AlbertTokenizer.from_pretrained() with the path to a single file or url is deprecated
I0512 18:29:05.415167 139756695570176 tokenization_utils.py:502] loading file transformers_vocab/albert-large-v2-spiece.model


['*sigh*']
 *sigh*


In [102]:
tokenizer = AlbertTokenizer.from_pretrained(
            pretrained_model_name_or_path="transformers_vocab/albert-large-v2-spiece.model",
            lowercase=True,
            add_prefix_space=True
        )

def get_prediction_v3(tweet, selected_text):
    
    tweet = " ".join(tweet.split())
    selected_text = " ".join(selected_text.split())
    
    len_st = len(selected_text)
    idx0 = None
    idx1 = None
    
    # get char idx
    for ind in (i for i, e in enumerate(tweet) if e == selected_text[0]):
        if tweet[ind: ind + len_st] == selected_text:
            idx0 = ind
            idx1 = ind + len_st - 1
            break
    
    # get char mask
    char_targets = [0] * len(tweet)
    if idx0 != None and idx1 != None:
        for ct in range(idx0, idx1 + 1):
            char_targets[ct] = 1
    
    # get word offsets
    tweet_offsets = []
    cursor = 0
    
    for word in tweet.split():

        encoded_word = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(word))
        number_of_tokens = len(encoded_word)
        
        start_offsets = cursor
        cursor += len(word)
        end_offsets = cursor
        
        for i in range(number_of_tokens):
            tweet_offsets.append((start_offsets, end_offsets))
            
        cursor += 1
    
    # get word idx
    target_idx = []
    for j, (offset1, offset2) in enumerate(tweet_offsets):
        
        if sum(char_targets[offset1: offset2]) > 0:
            target_idx.append(j)
    
    targets_start = target_idx[0]
    targets_end = target_idx[-1]
    
            
    input_ids_orig = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(tweet))
    
    return tweet[tweet_offsets[targets_start][0]: tweet_offsets[targets_end][1]]

# tweet = "S`ok, trying to plot alternatives as we speak *sigh*"
# selected_text = "*sigh*"
# get_prediction_v3(tweet, selected_text)

I0512 19:37:48.308559 139756695570176 tokenization_utils.py:420] Model name 'transformers_vocab/albert-large-v2-spiece.model' not found in model shortcut name list (albert-base-v1, albert-large-v1, albert-xlarge-v1, albert-xxlarge-v1, albert-base-v2, albert-large-v2, albert-xlarge-v2, albert-xxlarge-v2). Assuming 'transformers_vocab/albert-large-v2-spiece.model' is a path, a model identifier, or url to a directory containing tokenizer files.
W0512 19:37:48.309523 139756695570176 tokenization_utils.py:432] Calling AlbertTokenizer.from_pretrained() with the path to a single file or url is deprecated
I0512 19:37:48.309928 139756695570176 tokenization_utils.py:502] loading file transformers_vocab/albert-large-v2-spiece.model


In [16]:
train = pd.read_csv("/media/jionie/my_disk/Kaggle/Tweet/input/tweet-sentiment-extraction/train.csv")

In [103]:
for i in range(1000):
    print(
          train["selected_text"].iloc[i], \
          "-------------------", \
          get_prediction_v3(train["text"].iloc[i], train["selected_text"].iloc[i]))

I`d have responded, if I were going ------------------- I`d have responded, if I were going
Sooo SAD ------------------- Sooo SAD
bullying me ------------------- bullying me...
leave me alone ------------------- leave me alone
Sons of ****, ------------------- Sons of ****,
http://www.dothebouncy.com/smf - some shameless plugging for the best Rangers forum on earth ------------------- http://www.dothebouncy.com/smf - some shameless plugging for the best Rangers forum on earth
fun ------------------- fun
Soooo high ------------------- Soooo high
Both of you ------------------- Both of you
Wow... u just became cooler. ------------------- Wow... u just became cooler.
as much as i love to be hopeful, i reckon the chances are minimal =P i`m never gonna get my cake and stuff ------------------- as much as i love to be hopeful, i reckon the chances are minimal =P i`m never gonna get my cake and stuff
like ------------------- like
DANGERously ------------------- DANGERously
lost --------------

AttributeError: 'float' object has no attribute 'split'

In [34]:
tweet = "S`ok, trying to plot alternatives as we speak *sigh*"
selected_text = "*sigh*"


selected_text_copy = "".join(selected_text.split())
tweet_copy = "".join(tweet.split())

len_st = len(selected_text_copy)
idx0 = None
idx1 = None

for ind in (i for i, e in enumerate(tweet_copy) if e == selected_text_copy[0]):
    if tweet_copy[ind: ind + len_st] == selected_text_copy:
        idx0 = ind
        idx1 = ind + len_st - 1
        break
        
char_targets = [0] * len(tweet_copy)
if idx0 != None and idx1 != None:
    for ct in range(idx0, idx1 + 1):
        char_targets[ct] = 1
        
input_ids_orig = tokenizer.encode(tweet)
print(input_ids_orig)
input_ids_orig = input_ids_orig[1:-1]

tweet_offsets = []
idx = 0
for t in input_ids_orig:
    w = tokenizer.decode([t])
#     print(w, (idx, idx + len(w)))
    tweet_offsets.append((idx, idx + len(w)))
    idx += len(w)
    
target_idx = []
for j, (offset1, offset2) in enumerate(tweet_offsets):
    if sum(char_targets[offset1: offset2]) > 0:
        target_idx.append(j)
        
# print(tokenizer.decode(13))

targets_start = target_idx[0]
targets_end = target_idx[-1]

# ... i kinda lost respect  . i kinda lost respect
prediction = ""
for ix in range(targets_start, targets_end + 1):
    prediction += tweet_copy[tweet_offsets[ix][0]: tweet_offsets[ix][1]]
    if (ix + 1) < len(tweet_offsets) and tweet_offsets[ix][1] < tweet_offsets[ix + 1][0]:
        prediction += " "
        
len_prediction = len(prediction)

for ind in (i for i, e in enumerate(tweet) if e == prediction[0]):
    
    tweet_sub_sentence = "".join(tweet[ind:].split())
    
    if tweet_sub_sentence[:len_prediction] == prediction:
        orig_idx0 = ind
        break
        
for ind in range(orig_idx0, len(tweet)):

    if "".join(tweet[orig_idx0: ind+1].split()) == prediction:
        filtered_output = tweet[orig_idx0: ind+1]
        break

print(selected_text, filtered_output, prediction, tokenizer.decode(input_ids_orig[targets_start: targets_end+1]))

[2, 13, 18, 1, 3085, 15, 749, 20, 3798, 2676, 18, 28, 95, 1960, 1637, 18, 6872, 2483, 3]
*sigh* k *sigh* k*sigh* speak *s


In [35]:
tokenizer.tokenize(tweet)

['▁',
 's',
 '`',
 'ok',
 ',',
 '▁trying',
 '▁to',
 '▁plot',
 '▁alternative',
 's',
 '▁as',
 '▁we',
 '▁speak',
 '▁*',
 's',
 'igh',
 '*']

In [70]:
encoded = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(tweet))
encoded

[13,
 18,
 1,
 3085,
 15,
 749,
 20,
 3798,
 2676,
 18,
 28,
 95,
 1960,
 1637,
 18,
 6872,
 2483]

In [39]:
decoded = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(encoded))
decoded

's<unk>ok, trying to plot alternatives as we speak *sigh*'

In [41]:
print(len(encoded), len(tokenizer.tokenize(tweet)))

17 17


In [44]:
tweet.split()

['S`ok,',
 'trying',
 'to',
 'plot',
 'alternatives',
 'as',
 'we',
 'speak',
 '*sigh*']

In [43]:
for (i, word) in enumerate(tweet.split()):
        
    sub_tokens = tokenizer.tokenize(word)

    print(sub_tokens)

['▁', 's', '`', 'ok', ',']
['▁trying']
['▁to']
['▁plot']
['▁alternative', 's']
['▁as']
['▁we']
['▁speak']
['▁*', 's', 'igh', '*']
